# A.1 Explore the data and choose the KPI’s

In this notebook, we will explore the available datasets, choose 3 and then identify key performance indicators (KPIs) that can help us understand the business better.

In [ ]:
import os
import sys

import polars as pl

from pathlib import Path

sys.path.append(os.path.abspath(os.path.join("..")))

from utils.eda_dashboard import inspect_dataframe_interactive

In [2]:
DATA_PATH = Path("../../data/")

# Individual datasets
sets = {
    "cs": "cultural-sites",
    "ide": "idealista",
    "in": "incidences",
    "inc": "income",
    "lt": "lookup_tables",
    "po": "price_opendata",
    "un": "unemployment",
}

In [3]:
def read_many(path: Path | str) -> pl.DataFrame:
    """
    Read and concatenate multiple CSV or JSON files from a directory.

    The function automatically detects the file type (CSV or JSON) in the specified
    directory and processes all files of that type. The directory must contain
    only one file type - mixing CSV and JSON files will raise an error.

    Args:
        path: Path to the directory containing the files to read.
              Can be a string or Path object.

    Returns:
        A concatenated Polars DataFrame containing data from all files.
        Uses diagonal_relaxed concatenation to handle schema differences.

    Raises:
        ValueError: If the directory contains both CSV and JSON files,
                   or if no CSV/JSON files are found.

    Example:
        >>> df = read_many("/path/to/csv/files")
        >>> df = read_many(Path("/path/to/json/files"))
    """
    f_path = Path(path)

    # Get all CSV and JSON files
    csv_files = list(f_path.glob("*.csv"))
    json_files = list(f_path.glob("*.json"))

    # Validate that folder contains only one file type
    if csv_files and json_files:
        raise ValueError(
            "Folder contains both CSV and JSON files. Only one file type is supported."
        )

    if not csv_files and not json_files:
        raise ValueError("No CSV or JSON files found in the specified folder.")

    # Process based on file type found
    if csv_files:
        # CSV logic
        print(f"Reading {len(csv_files)} CSV file(s)")
        queries = [pl.scan_csv(file) for file in csv_files]
        dataframes = pl.collect_all(queries)
    else:
        # JSON logic with error handling
        print(f"Reading {len(json_files)} JSON file(s)")
        dataframes = []
        for file in json_files:
            try:
                # Try standard JSON first
                df = pl.read_json(file, infer_schema_length=10000)
                dataframes.append(df)
            except pl.ComputeError:
                try:
                    # If that fails, try JSONL/NDJSON format
                    df = pl.read_ndjson(file, infer_schema_length=10000)
                    dataframes.append(df)
                except Exception as e:
                    raise ValueError(f"Failed to read JSON file {file}: {e}")

    return pl.concat(dataframes, how="diagonal_relaxed")

## Cultural sites

In [4]:
df_cs = read_many(DATA_PATH / sets["cs"])
_ = inspect_dataframe_interactive(df_cs, name="Cultural Sites")

Reading 1 CSV file(s)
📊 CULTURAL SITES - SUMMARY
Shape: 871 rows × 32 columns
Memory usage: 0.24 MB

Column Types:
  Numeric: 14
  Categorical: 16
  Date: 0
  Other: 2

Data Quality:
  Missing values: 7,705 (27.64%)
  Duplicate rows: 0 (0.00%)

💡 Key Insights:
  ⚠️  High missing data rate (27.6%)



## Idealista

In [5]:
df_ide = read_many(DATA_PATH / sets["ide"])
_ = inspect_dataframe_interactive(df_ide, name="Idealista")

Reading 132 JSON file(s)
📊 IDEALISTA - SUMMARY
Shape: 21,389 rows × 37 columns
Memory usage: 7.54 MB

Column Types:
  Numeric: 8
  Categorical: 15
  Date: 0
  Other: 14

Data Quality:
  Missing values: 55,569 (7.02%)
  Duplicate rows: 1,254 (5.86%)

💡 Key Insights:
  ⚠️  Significant duplicates (5.9%)



## Incidences

In [6]:
df_in = read_many(DATA_PATH / sets["in"])
_ = inspect_dataframe_interactive(df_in, name="Incidences")

Reading 9 CSV file(s)
📊 INCIDENCES - SUMMARY
Shape: 900 rows × 26 columns
Memory usage: 0.21 MB

Column Types:
  Numeric: 14
  Categorical: 12
  Date: 0
  Other: 0

Data Quality:
  Missing values: 5,596 (23.91%)
  Duplicate rows: 0 (0.00%)

💡 Key Insights:
  ⚠️  High missing data rate (23.9%)



## Income

In [7]:
df_inc = read_many(DATA_PATH / sets["inc"])
_ = inspect_dataframe_interactive(df_inc, name="Income")

Reading 11 CSV file(s)
📊 INCOME - SUMMARY
Shape: 811 rows × 7 columns
Memory usage: 0.05 MB

Column Types:
  Numeric: 4
  Categorical: 3
  Date: 0
  Other: 0

Data Quality:
  Missing values: 0 (0.00%)
  Duplicate rows: 0 (0.00%)

✅ Data appears to be in good shape!



## Lookup tables

In [8]:
df_lt = read_many(DATA_PATH / sets["lt"])
_ = inspect_dataframe_interactive(df_lt, name="Lookup Table")

Reading 2 CSV file(s)
📊 LOOKUP TABLE - SUMMARY
Shape: 127 rows × 8 columns
Memory usage: 0.01 MB

Column Types:
  Numeric: 0
  Categorical: 8
  Date: 0
  Other: 0

Data Quality:
  Missing values: 0 (0.00%)
  Duplicate rows: 14 (11.02%)

💡 Key Insights:
  ⚠️  Significant duplicates (11.0%)
  ℹ️  No numeric columns found



## Price opendata

In [9]:
df_po = read_many(DATA_PATH / sets["po"])
_ = inspect_dataframe_interactive(df_po, name="Price Opendata")

Reading 1 JSON file(s)
📊 PRICE OPENDATA - SUMMARY
Shape: 73 rows × 5 columns
Memory usage: 0.02 MB

Column Types:
  Numeric: 2
  Categorical: 2
  Date: 0
  Other: 1

Data Quality:
  Missing values: 0 (0.00%)
  Duplicate rows: 0 (0.00%)

💡 Key Insights:
  ⚠️  Small dataset



C:\Users\mmpei\AppData\Local\Temp\ipykernel_17276\1472380360.py:55: DeprecationWarning:

accessing `ComputeError` from the top-level `polars` module was deprecated in version 1.0.0. Import it directly from the `polars.exceptions` module instead, e.g.: `from polars.exceptions import ComputeError`



## Unemployment

In [10]:
df_un = read_many(DATA_PATH / sets["un"])
_ = inspect_dataframe_interactive(df_un, name="Unemployment")

Reading 13 JSON file(s)
📊 UNEMPLOYMENT - SUMMARY
Shape: 13 rows × 4 columns
Memory usage: 0.08 MB

Column Types:
  Numeric: 0
  Categorical: 1
  Date: 0
  Other: 3

Data Quality:
  Missing values: 13 (25.00%)
  Duplicate rows: 0 (0.00%)

💡 Key Insights:
  ⚠️  High missing data rate (25.0%)
  ℹ️  No numeric columns found
  ⚠️  Small dataset

